In [1]:
!pip install openai


In [ ]:
import os
from openai import OpenAI

api_key = os.getenv("OPENAI_API_KEY")
#print(f"API Key:{api_key}")
openai_client = OpenAI(
    base_url="https://api.openai.com/v1",
    api_key=api_key#enter api key
)

In [3]:
import pandas as pd

In [4]:
# get the csv file
file_path = 'student_profiles.csv'

# read the csv file
df = pd.read_csv(file_path)

In [11]:
df.head()

,field,major,bio
0,STEM,Computer Science,A dedicated student with expertise in object-o...
1,STEM,Computer Science,A dedicated and enthusiastic Computer Science ...
2,STEM,Computer Science,A dedicated student with a keen interest in so...
3,STEM,Computer Science,A dedicated student with a strong background i...
4,STEM,Computer Science,An aspiring computer scientist with a passion ...


In [5]:
row1 = df.iloc[0]


In [ ]:
prompt = """
<|im_start|>system
Evaluate the student profile objectively using this JSON schema:

<schema>
{{
    "overall_score": "number",
    "Academics_and_research": "number",
    "Professional_experience": "number", 
    "Leadership_and_Initiative": "number",
    "Depth_of_experience": "number",
}}
</schema>

Scoring Guidelines:
- Maximum typical score is 100
- Scores above 85 are EXTREMELY rare
- 50 represents a solid, competitive student profile
- Scoring criteria:
  - Be more critical and conservative
  - Scrutinize each achievement's substantiveness
  - Avoid grade inflation
  - Emphasize quality over quantity

Scoring Constraints:
- Rarely award more than 85 points
- Most students should score between 45-60
- Exceptional achievements warrant careful consideration
- Penalize vague or unsubstantiated claims
<|im_end|>

<|im_start|>user
Evaluate the following {major} student profile:
{student_profile}
<|im_end|>

<|im_start|>assistant
"""

In [19]:
import json
def invoke_openai(prompt, model = 'gpt-4o-mini'):
    for i in range(3):
        try:
            kwargs = {
                    "model": model,
                    "messages": [{"role": "user", "content": prompt}],
                    "response_format": {"type": "json_object"}
                }

            response = openai_client.chat.completions.create(**kwargs)
            json_response = json.loads(response.choices[0].message.content)
            return json_response
        except Exception as e:
            print(f"Attempt {i + 1} failed with error: {e}")
            if i == 2:
                print("Failed all attempts. Exiting...")
                return None

In [23]:
prompt_formatted = prompt.format(major=row1['major'], student_profile=row1['bio'])
response = invoke_openai(prompt_formatted)

In [24]:
import json

response

{'overall_score': 65,
 'Academics_and_research': 60,
 'Professional_experience': 70,
 'Leadership and Initiative': 65,
 'Depth_of_experience': 65}

In [25]:
results = []
for i, row in df.iterrows():
    prompt_formatted = prompt.format(major=row['major'], student_profile=row['bio'])
    response = invoke_openai(prompt_formatted)
    results.append(response)

    if (i + 1) % 100 == 0:
        print(f"Processed {i} rows")

Processed 99 rows
Processed 199 rows
Processed 299 rows
Processed 399 rows
Processed 499 rows
Processed 599 rows
Processed 699 rows
Processed 799 rows
Processed 899 rows
Processed 999 rows
Processed 1099 rows
Processed 1199 rows
Processed 1299 rows
Processed 1399 rows
Processed 1499 rows
Processed 1599 rows
Processed 1699 rows


In [26]:
score_df = df.copy()
scores = pd.DataFrame(results)

score_df = pd.concat([score_df, scores], axis=1)

In [27]:
# Save the scores to a new csv file
score_df.to_csv('student_profiles_scored.csv', index=False)